# Part 1: Stand/Sit Detector
### This part creates a stand/sit detector based on images of sheep and uses accelerometer data as ground truth

In [3]:
# imports
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from tensorflow.keras import datasets, layers, models

In [4]:
# read in the data first
files = ['W.csv', 'B.csv', 'M.csv', 'P.csv', 'O_0.csv', 'N.csv', 'K.csv', 'Q.csv']
sheep_raw = []
data_dir = 'data/'
for file in files:
    path = data_dir + file
    df = pd.read_csv(path)
    # drop unnecessary columns
    df = df.drop(columns=df.columns[3:])
    df = df.drop(columns=df.columns[0:1])

    # rename columns
    df.columns = ['File', 'Standing']

    # change accelerometer value to 1 if standing, 0 if sitting
    df['Standing'] = np.where(df['Standing'] < -0.4, 1, 0)

    # change the timestamp to the filename for the image
    time_stamps = df['File'].to_list()
    file_names = []
    for ts in time_stamps:
        filename = ts
        filename = filename.replace(' ', '_')
        filename = filename.replace(':', ',')
        filename = filename.replace('/', '-')
        filename = filename + '.png'

        # UNCOMMENT THIS LINE WHEN YOU KNOW PATH TO IMAGE FOLDER
        #filename = PATH_TO_IMAGES + '/' + file[0] + '/' + filename

        file_names.append(filename)

    df = df.assign(File=file_names)
    sheep_raw.append(df)

In [11]:
sheep_raw[0].to_csv('test.csv', index=False)

In [6]:
# I suspect there will be too many images here and this will cause a stack overflow, but let's try it
X = []
y = []

bad_reads = 0
for sheep in sheep_raw:
    files = sheep['File'].to_list()
    labels = sheep['Standing'].to_list()
    for file, label in zip(files, labels):
        try:
            img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)

            # downscale image
            img = cv2.resize(img, (200,200))

            # normalize image
            img = img/255
            # img = cv2.normalize(img, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

            # store image
            X.append(img)
            y.append(label)
        except:
            bad_reads += 1
            continue

print(f'Bad or missing images: {bad_reads}')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

Bad or missing images: 154779


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [9]:
def create_model(num_filters: list, kernel_sizes: list, input_shape: tuple):
    model = models.Sequential()

    model.add(layers.Conv2D(num_filters[0], (kernel_sizes[0],kernel_sizes[0]), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2,2)))

    for i in range(1, len(kernel_sizes)):
        model.add(layers.Conv2D(num_filters[i], (kernel_sizes[i],kernel_sizes[i]), activation='relu'))
        model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))

    model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics=['accuracy'])
    print(model.summary())

    return model

In [10]:
params = {
    'num_filters': [64, 32, 16],
    'kernel_sizes': [3, 3, 3]
}

model = create_model(params['num_filters'], params['kernel_sizes'], (200,200,1))

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 198, 198, 64)      640       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 99, 99, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 97, 97, 32)        18464     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 48, 48, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 46, 46, 16)        4624      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 23, 23, 16)      